# Multithreaded Monte Carlo approach to estimating π

The aim of this notebook is to compute the value of $\pi$ using a parallel Monte-Carlo implementation using thread-based parallelism.

It follows the classic approach to consider a circle of radius $1$ inscribed inside a unit square with side length $2$, running from $-1$ to $1$ in each axis. Since the area of the circle is $\pi$, but the area of the square is $4$, a random "dart" thrown to the square will be inside the square with probability $\pi/4$. Therefore if we throw $N$ darts randomly, roughly $M = N \pi/4$ will be inside the unit circle:

In [ ]:
using Plots
using Distributions

# Determine where 500 random darts would land
N  = 500
d  = Uniform(-1.0, 1.0)
xs = rand(d, N)
ys = rand(d, N)
is_inside = [sqrt(x^2 + y^2) ≤ 1.0 for (x, y) in zip(xs, ys)]
M = sum(is_inside)

# Plot the circle
circle = Plots.partialcircle(0, 2π, 100)
p = plot(circle; title="Inside: M = $M => π ≈ $(4M/N)",
         aspect_ratio=:equal, legend=false, grid=false,
         xlims=(-1, 1), lw=3)

# Plot the points
scatter!(p, xs, ys; ms=2, color=[red ? :red : :black for red in is_inside])

## Basic Julia implementation

A basic Julia implementation of this method to compute $\pi$ is:

In [ ]:
function montecarlo_pi(N)
    M = 0  # count darts that landed in the circle
    for i in 1:N
        if sqrt(rand()^2 + rand()^2) < 1.0
            M += 1
        end
    end
    4 * M / N
end

In [ ]:
montecarlo_pi(10_000_000)

**Exercise:**

1. Write a function `montecarlo_pi_threads(N::Int)`, which is based on `montecarlo_pi(N::Int)`, but distributes the work using the `Threads.nthreads()` available threads.


2. Benchmark and compare both `montecarlo_pi(N::Int)` and `montecarlo_pi_threads(N::Int)`. For this part (and all following parts) use `N = 10_000_000` as a reasonable value for $N$.


3. Based on the function `montecarlo_pi(N::Int)` code up a function `montecarlo_pi_all(Ns::Vector{Int})`, which computes $\pi$ for all passed values for $N$. The function should be serial.


4. Write a function `montecarlo_pi_all_parallel(Ns::Vector{Int})`, which uses multithreading to do the same thing as 3., but in parallel. Build this function upon `montecarlo_pi(N::Int)` as well. Benchmark and compare this function with the implementation from 3.

5. Calculate $\pi$ estimates for
   ```julia
   Ns = @. ceil(Int, exp10(1:0.15:8.1))
   ```
   and plot $\pi$ versus $N$ on a semilog plot.